# 이상탐지
- (Prophet) https://www.analyticsvidhya.com/blog/2021/12/anomaly-detection-model-using-facebook-prophet/
- https://mkang32.github.io/python/2020/12/15/prophet-intro.html
- (sklearn) https://github.com/uiandwe/TIL/blob/master/Machine_Running/%EC%9D%B4%EC%83%81%EA%B0%90%EC%A7%80/Anomaly_Detection_using_Isolation_Forest.ipynb
- https://john-analyst.medium.com/isolation-forest%EB%A5%BC-%ED%86%B5%ED%95%9C-%EC%9D%B4%EC%83%81%ED%83%90%EC%A7%80-%EB%AA%A8%EB%8D%B8-9b10b43eb4ac
- (RNN) https://velog.io/@jhpark/RNN-%EC%8B%9C%EA%B3%84%EC%97%B4-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%9D%B4%EC%83%81%EC%B9%98-%ED%83%90%EC%A7%80-%EC%BD%94%EB%93%9C-%EB%A6%AC%EB%B7%B0

## !!!!!train은 어제까지로 바꾸고, forecast에 어제부터로 해야하는거 아닌지??

In [ ]:
from datetime import timedelta
df_anomaly = final_df[['날짜']]
col_na = []

final_df = final_df[final_df.columns[final_df.apply(lambda x: len(x[x.notnull().values]) > 30).values == True]] # 값이 30개 이상인 항목만
for search_key in tqdm(final_df.columns[1:]): #'날짜'제외
    new_df = final_df[['날짜', search_key]]
    new_df = new_df.set_index('날짜')
    new_df_prophet = new_df.reset_index()[['날짜', new_df.columns[0]]].rename({'날짜':'ds', new_df.columns[0]:'y'}, axis='columns')
    m = Prophet()#changepoint_range=0.95, changepoint_prior_scale=0.05) #holidays=holidays)
    m.add_country_holidays(country_name='KR')
    m.fit(new_df_prophet)
    future = m.make_future_dataframe(periods=365, freq='D')
    forecast = m.predict(future)
    forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)
    result = pd.concat([new_df_prophet.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
    result['error'] = result['y'] - result['yhat'] #실제값 - 예측값범위
    result['uncertainty'] = result['yhat_upper'] - result['yhat_lower'] #범위, 불확실성
    # 본래 abs를 붙여 낮게 나온 이상치도 출력하는 코드였지만, 검색어 특성상 높게 나온 지점만 산출하기 위해 abs 삭제
    #비율에 따라, 검색량이 절대적으로 적은(일정 수치 이하) 경우는 가중치를 바꾸도록??
    result['anomaly'] = result.apply(lambda x: 'Yes' if((x['error']) > 1*x['uncertainty']) else 'No', axis = 1) #원래는 1.5. 나는 0.4
    #result_new = result[result['anomaly']=='Yes']
    #anomaly_list = list(result_new.index)
    df_temp = result['anomaly'].reset_index().rename(columns={'ds':'날짜','anomaly':search_key})
    df_anomaly = pd.merge(df_anomaly,df_temp, on = '날짜')
df_anomaly

In [ ]:
# 어제 이상치 추출
from datetime import datetime
df_anomaly_ = df_anomaly.set_index('날짜')
df_anomaly_.T.replace('No', np.NaN).count().tail() #날짜
day_num = 1
anomaly_yesterday = df_anomaly_.T[str(datetime.now().date() - relativedelta(days=day_num))][df_anomaly_.T[str(datetime.now().date() - relativedelta(days=day_num))].values == 'Yes'].keys() #어제 이상치..

In [ ]:
df_anomaly_.T.replace('No', np.NaN).count().tail(7)

In [ ]:
anomaly_yesterday

In [ ]:
weight = 1.5 # 오차 허용범위(높을 수록 둔감)

for search_key in anomaly_yesterday:
    new_df = final_df[['날짜', search_key]]
    new_df = new_df.set_index('날짜')
    new_df_prophet = new_df.reset_index()[['날짜', new_df.columns[0]]].rename({'날짜':'ds', new_df.columns[0]:'y'}, axis='columns')
    m = Prophet()#changepoint_range=0.95, changepoint_prior_scale=0.05) #holidays=holidays)
    m.add_country_holidays(country_name='KR')
    m.fit(new_df_prophet)
    future = m.make_future_dataframe(periods=365, freq='D')
    forecast = m.predict(future)
    forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)
    result = pd.concat([new_df_prophet.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
    result['error'] = result['y'] - result['yhat'] #실제값 - 예측값범위
    result['uncertainty'] = result['yhat_upper'] - result['yhat_lower'] #범위, 불확실성
    # 본래 abs를 붙여 낮게 나온 이상치도 출력하는 코드였지만, 검색어 특성상 높게 나온 지점만 산출하기 위해 abs 삭제
    #비율에 따라, 검색량이 절대적으로 적은(일정 수치 이하) 경우는 가중치를 바꾸도록??
    result['anomaly'] = result.apply(lambda x: 'Yes' if((x['error']) > weight*x['uncertainty']) else 'No', axis = 1) #원래는 1.5. 나는 0.4
    #result_new = result[result['anomaly']=='Yes']
    #visualize the anomaly data
    fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', title=new_df.columns[0])
    #slider
    fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                  dict(count=1, label='1y', step="year", stepmode="backward"),
                  dict(count=2, label='3y', step="year", stepmode="backward"),
                  dict(count=2, label='5y', step="year", stepmode="backward"),
                  dict(step="all")
            ])
        )
    )
    fig.show()

In [ ]:
# 외부링크

In [ ]:
from bokeh.plotting import figure, output_file, show
p = figure(title=new_df.columns[0], x_axis_label = 'ds', y_axis_label = 'y')
p.circle(result.reset_index()['ds'],[result[result.reset_index()['anomaly'].values == 'No']['y'], result[result.reset_index()['anomaly'].values == 'Yes']['y']], color=['blue','red'])

#p.circle(result.reset_index()['ds'],result[result.reset_index()['anomaly'].values == 'No']['y'], color='blue')
#p.scatter(result.reset_index()['ds'],result[result.reset_index()['anomaly'].values == 'Yes']['y'], color='red')
show(p)

In [ ]:
import plotly.express as px
fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', title=new_df.columns[0])
#slider
fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
              dict(count=1, label='1y', step="year", stepmode="backward"),
              dict(count=2, label='3y', step="year", stepmode="backward"),
              dict(count=2, label='5y', step="year", stepmode="backward"),
              dict(step="all")
        ])
    )
)
fig.show()
fig.write_html("a.html")

In [ ]:
# 카카오 API
# -https://developers.kakao.com/tool/rest-api/open/post/v2-api-talk-memo-default-send
import json
import requests

url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
# 사용자 토큰
headers = {
    "Authorization": "Bearer " + _cfg['Kakao_Authorization']
}
data = {
    "template_object" : json.dumps({ "object_type" : "text",
                                     "text" : f'{str(datetime.now().date() - relativedelta(days=1))} 이상검색어 목록입니다. {list(anomaly_yesterday)}',
                                     "link" : {
                                                 "web_url" : "www.naver.com",
                                                 "mobile_web_url": "https://developers.kakao.com"
                                              }
    })
}
response = requests.post(url, headers=headers, data=data)
print(response.status_code)
if response.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))
    # 이미지 보내는 방법 추가 필요

# 이메일
- https://www.infoking.site/138

In [ ]:
import smtplib
from email.message import EmailMessage

# STMP 서버의 url과 port 번호
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 465

# 1. SMTP 서버 연결
smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)

EMAIL_ADDR = config.ID
EMAIL_PASSWORD = config.PASSWORD

# 2. SMTP 서버에 로그인
smtp.login(EMAIL_ADDR, EMAIL_PASSWORD)

# 3. MIME 형태의 이메일 메세지 작성
message = EmailMessage()
message.set_content('이메일 본문')
message["Subject"] = "이메일 제목"
message["From"] = EMAIL_ADDR  #보내는 사람의 이메일 계정
message["To"] = '받는 사람의 이메일 계정'

# 4. 서버로 메일 보내기
smtp.send_message(message)

# 5. 메일을 보내면 서버와의 연결 끊기
smtp.quit()

In [ ]:
# 사진첨부
import imghdr #이미지 첨부를 위한 라이브러리
import smtplib
from email.message import EmailMessage

# STMP 서버의 url과 port 번호
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 465

# 1. SMTP 서버 연결
smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)

EMAIL_ADDR = '본인의 이메일 계정'
EMAIL_PASSWORD = '본인의 이메일 계정 비밀번호'

# 2. SMTP 서버에 로그인
smtp.login(EMAIL_ADDR, EMAIL_PASSWORD)

# 3. MIME 형태의 이메일 메세지 작성
message = EmailMessage()
message.set_content('이메일 본문')
message["Subject"] = "이메일 제목"
message["From"] = EMAIL_ADDR  #보내는 사람의 이메일 계정
message["To"] = '받는 사람의 이메일 계정'

# 3-1. 이메일에 사진 첨부하기
with open('사진경로', 'rb') as image:
  image_file = image.read() # 이미지 파일 읽어오기

image_type = imghdr.what('e-mail', image_file)
message.add_attachment(image_file, maintype = 'image', subtype = image_type)


# 4. 서버로 메일 보내기
smtp.send_message(message)

# 5. 메일을 보내면 서버와의 연결 끊기
smtp.quit()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

#검색 키워드 설정
search_key = '과당'
new_df = final_df[['날짜', search_key]]
new_df.shape

In [ ]:
#df['timestamp'] = pd.to_datetime(df['timestamp'])
#df = df.set_index('timestamp').resample('H').mean().reset_index()
new_df = new_df.set_index('날짜')
new_df.shape

In [ ]:
#express to plot entire data
fig = px.line(new_df.reset_index(), x='날짜', y=new_df.columns[0], title=new_df.columns[0])
#slider
fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
              dict(count=1, label='1y', step="year", stepmode="backward"),
              dict(count=2, label='2y', step="year", stepmode="backward"),
              dict(count=5, label='5y', step="year", stepmode="backward")
        ])
    )
)
fig.show()

In [ ]:
from prophet import Prophet
new_df_prophet = new_df.reset_index()[['날짜', new_df.columns[0]]].rename({'날짜':'ds', new_df.columns[0]:'y'}, axis='columns')
# 상한하한 (그러나 전체적으로 예측치가 올라가 이상탐지에 적합하지않아서 제외)
#new_df_prophet['cap']=train.max()['y']
#new_df_prophet['floor']=0

#train test split
train = new_df[(new_df['ds'] >= '2016-01-01') & (new_df['ds'] <= '2021-12-31')]
test = new_df[(new_df['ds'] >= '2022-01-01')]

In [ ]:
#Prophet(holidays=holidays로 추가 가능)
event1 = pd.DataFrame({
  'holiday': 'event1',
  'ds': pd.to_datetime(['2017-08-15', '2017-08-16']),#, "2018-08-14", '2018-08-15', '2018-08-16']),
  'lower_window': 0,
  'upper_window': 120,
})
"""
event2 = pd.DataFrame({
  'holiday': 'event2',
  'ds': pd.to_datetime(["2018-10-25", "2018-10-24"]),
  'lower_window': 0,
  'upper_window': 1,
})
"""
holidays = event1
#holidays = pd.concat((event1, event2))

In [ ]:
m = Prophet()#(growth='logistic'--상하한)#changepoint_range=0.95, changepoint_prior_scale=0.05) #holidays=holidays)
m.add_country_holidays(country_name='KR')
m.fit(new_df_prophet) # train test 분리 안 하고 전체 넣어도 될 듯? (원래는 train이였음)-2022년에 사건이 있는 경우 train으로 하는게 좋음

In [ ]:
future = m.make_future_dataframe(periods=365, freq='D')
#future['cap']=500
#future['floor']=0 #하한지정

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)

In [ ]:
forecast[['ds', 'holidays']].sort_values(by='holidays', ascending=False)#[forecast['ds'] == '2017-11-11']

In [ ]:
from prophet.plot import add_changepoints_to_plot
result = pd.concat([new_df_prophet.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
fig1 = m.plot(forecast)
add_changepoints_to_plot(fig1.gca(), m, forecast)

In [ ]:
comp = m.plot_components(forecast)

In [ ]:
result['error'] = result['y'] - result['yhat'] #실제값 - 예측값범위
result['uncertainty'] = result['yhat_upper'] - result['yhat_lower'] #범위, 불확실성

In [ ]:
weight = 1.5
# 본래 abs를 붙여 낮게 나온 이상치도 출력하는 코드였지만, 검색어 특성상 높게 나온 지점만 산출하기 위해 abs 삭제
#비율에 따라, 검색량이 절대적으로 적은(일정 수치 이하) 경우는 가중치를 바꾸도록??
result['anomaly'] = result.apply(lambda x: 'Yes' if((x['error']) > weight*x['uncertainty']) else 'No', axis = 1) #원래는 1.5. 나는 0.4
#result['anomaly'] = np.where(np.abs(x['error']) > 1.5*x['uncertainty']), 'Yes', 'No') #Alternate way

In [ ]:
result

In [ ]:
#visualize the anomaly data
fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', title=new_df.columns[0])
#slider
fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
              dict(count=1, label='1y', step="year", stepmode="backward"),
              dict(count=2, label='3y', step="year", stepmode="backward"),
              dict(count=2, label='5y', step="year", stepmode="backward"),
              dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
result_new = result[result['anomaly']=='Yes']
result_new
#result_new.sort_values(by='error', ascending=False).head(50)

In [ ]:
from datetime import timedelta
anomaly_list = list(result_new.index)
single_day = []
multiple_days = []
for i in anomaly_list:
    if (i+timedelta(days=1) in anomaly_list) or (i-timedelta(days=1) in anomaly_list):
        multiple_days.append(i)
    else:
        single_day.append(i)
temp_list = []
for i in multiple_days:
    if (i+timedelta(days=1) in anomaly_list) and (i-timedelta(days=1) in anomaly_list):
        temp_list.append(i)
multiple_days = [x for x in multiple_days if x not in temp_list]

In [ ]:
#연속일자
if multiple_days != []:
    print('시작일: ', multiple_days[0]) # 시작일. i%2==0
    print('종료일: ', multiple_days[1]) # 종료일. i%2==1

In [ ]:
multiple_days

In [ ]:
# 단독일자
single_day

# 예측 평가??
- https://dining-developer.tistory.com/25

In [ ]:
new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]

In [ ]:
import datetime
# 모델 생성 후 학습
model = Prophet() #seasonality_mode='multiplicative' 승법기법??
model.add_country_holidays(country_name='KR')
# 마지막 12개월 제외시키기
#train = new_df_prophet[(new_df_prophet['ds'] >= '2016-01-01') & (new_df_prophet['ds'] <= '2021-12-31')] 
y_true = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['y'].values

# 모델 학습
model.fit(new_df_prophet) #학습모델로 평가하는게 맞나?

# train set 마지막 1년 날짜 생성
last_1year = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['ds']
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

# 에측
forecast = model.predict(last_1year)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

model.plot(forecast)
plt.xlim([datetime.date(2022, 1, 1), datetime.date(2022, 11, 1)])
#plt.ylim([0, 30000])
plt.show()

In [ ]:
last_1year.reset_index()['ds']

In [ ]:
new_df_prophet[:-8]

In [ ]:
# 마지막 12개월 제외시키기
train = new_df_prophet[(new_df_prophet['ds'] >= '2016-01-01') & (new_df_prophet['ds'] <= '2021-12-31')] 
y_true = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['y'].values
# 모델 생성 후 학습
model = Prophet()
model.add_country_holidays(country_name='KR')
model.fit(new_df_prophet) #실제로는 new_df_prophet
# train set 마지막 1년 날짜 생성
last_1year = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['ds']
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

# 예측하고 비교하기
# 여기에서는 MAE를 살펴본다.
forecast = model.predict(last_1year)
y_pred = forecast['yhat'].values
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)*100
print('MAE: %.3f' % mae)
print('MAPE: %.3f' % mape)

In [ ]:
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
#plt.ylim([0, 20000])
plt.show()
# 2022년에 변동점이 있는 경우 예측을 아예 못 함(오히려 못해서 좋음!)

In [ ]:
pd.DataFrame(y_pred,y_true).reset_index().corr() #상관 정도에 따라 anomaly 가중치를 설정??(절대량,상관정도 2개의 기준으로 하는게 좋아보임~)

In [ ]:
# 교차검증
# - https://gmnam.tistory.com/230